<a href="https://colab.research.google.com/github/mrindani/How_Many_topics/blob/master/Wine_Reviews_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
pip install pyLDAvis

In [ ]:
# Usual imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
!pip install chart_studio

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go

In [ ]:
%matplotlib inline
import os
#print(os.listdir("../input"))

# Plotly based imports for visualization
from plotly import tools
#import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mrindani","key":"9928128026eff6e60530d47ec157f92e"}'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                    title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
shivan118/healthcare-analytics                         AV : Healthcare Analytics                           2MB  2020-09-13 17:40:05           2271        113  0.9411765        
datatattle/covid-19-nlp-text-classification            Coronavirus tweets NLP - Text Classification        4MB  2020-09-08 11:40:11           1609         86  1.0              
anmolkumar/health-insurance-cross-sell-prediction      Health Insurance Cross Sell Prediction 🏠 🏥          6MB  2020-09-11 18:39:31           2785        181  1.0              
Cornell-University/arxiv                               arXiv Dataset                                     889MB  202

In [ ]:
! kaggle datasets download -d zynicide/wine-reviews

 96% 49.0M/50.9M [00:02<00:00, 13.2MB/s]
100% 50.9M/50.9M [00:02<00:00, 19.5MB/s]


In [ ]:
! mkdir wine-reviews

! unzip wine-reviews.zip -d wine-reviews

Archive:  wine-reviews.zip
  inflating: wine-reviews/winemag-data-130k-v2.csv  
  inflating: wine-reviews/winemag-data-130k-v2.json  
  inflating: wine-reviews/winemag-data_first150k.csv  


In [ ]:
# Loading data
wines = pd.read_csv('/content/wine-reviews/winemag-data_first150k.csv')
wines.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [ ]:
# Creating a spaCy object
nlp = spacy.load('en_core_web_lg')

In [ ]:
doc = nlp(wines["description"][3])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [ ]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [ ]:
review = str(" ".join([i.lemma_ for i in doc]))

In [ ]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [ ]:
# POS tagging
for i in nlp(review):
    print(i,"=>",i.pos_)

this => DET
spend => VERB
20 => NUM
month => NOUN
in => ADP
30 => NUM
% => NOUN
new => ADJ
french => ADJ
oak => NOUN
, => PUNCT
and => CCONJ
incorporate => VERB
fruit => NOUN
from => ADP
Ponzi => PROPN
's => PART
Aurora => PROPN
, => PUNCT
Abetina => PROPN
and => CCONJ
Madrona => PROPN
vineyard => NOUN
, => PUNCT
among => ADP
other => ADJ
. => PUNCT
aromatic => ADJ
, => PUNCT
dense => ADJ
and => CCONJ
toasty => ADJ
, => PUNCT
-pron- => X
deftly => ADV
blend => VERB
aroma => NOUN
and => CCONJ
flavor => NOUN
of => ADP
toast => NOUN
, => PUNCT
cigar => PROPN
box => PROPN
, => PUNCT
blackberry => PROPN
, => PUNCT
black => ADJ
cherry => NOUN
, => PUNCT
coffee => NOUN
and => CCONJ
graphite => NOUN
. => PUNCT
tannin => NOUN
be => VERB
polish => ADJ
to => ADP
a => DET
fine => ADJ
sheen => NOUN
, => PUNCT
and => CCONJ
frame => VERB
a => DET
finish => NOUN
load => NOUN
with => ADP
dark => ADJ
chocolate => NOUN
and => CCONJ
espresso => NOUN
. => PUNCT
drink => VERB
now => ADV
through => ADP
2032 

In [ ]:
# Parser for reviews
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [ ]:
tqdm.pandas()
wines["processed_description"] = wines["description"].progress_apply(spacy_tokenizer)

100%|██████████| 150930/150930 [01:01<00:00, 2474.23it/s]


In [ ]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(wines["processed_description"])

In [ ]:
NUM_TOPICS = 10

In [ ]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [ ]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [ ]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('flavors', 18025.224734528314), ('oak', 13842.25675562436), ('fruit', 11005.816421345886), ('cherry', 7853.444661170265), ('blackberry', 7212.664936165386), ('tannins', 7209.087172705971), ('chocolate', 7124.983385902619), ('finish', 5446.193641575372), ('cherries', 5355.749068622052), ('tannic', 5341.089643199956)]
Topic 1:
[('green', 11616.470138415712), ('flavors', 8587.40195207835), ('palate', 8293.282037008383), ('aromas', 7709.27082126023), ('apple', 6415.136062575027), ('nose', 6034.275411806288), ('finish', 5954.10212157823), ('citrus', 5348.4921188029275), ('lime', 5229.540637184832), ('fresh', 5220.739871322384)]
Topic 2:
[('vineyard', 5438.345382683172), ('oak', 5196.076415854352), ('chardonnay', 4988.167292140623), ('best', 3719.3648458501707), ('new', 3223.510864909686), ('valley', 2687.9010921093077), ('vanilla', 2271.4857192292075), ('bottling', 2123.874595733775), ('pie', 2039.8578588477822), ('buttered', 2015.0960495474424)]
Topic 3:
[('blend', 95

In [ ]:
# Transforming an individual sentence
text = spacy_tokenizer("Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.0050002  0.00500178 0.00500075 0.00500001 0.3576935  0.13095929
 0.00500038 0.47634377 0.0050003  0.00500002]


In [ ]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952   8.115230       1        1  16.742796
4       0.648955  71.707100       2        1  14.126033
8      71.019684   9.971006       3        1  12.957978
0       3.785981  33.185135       4        1  12.807490
6     -42.666500  -9.222070       5        1  10.099722
5      -3.621689 -43.526516       6        1   9.543020
1      -3.632402  -4.171708       7        1   8.429452
2     -36.584400  35.804413       8        1   5.290853
3      44.391380  50.615383       9        1   5.011289
9      40.235809 -31.591665      10        1   4.991367, topic_info=             Term          Freq         Total Category  logprob  loglift
13490        wine  91232.000000  91232.000000  Default  30.0000  30.0000
11930       sweet  23002.000000  23002.000000  Default  29.0000  29.0000
8150          oak  20990.000000  20990.000000  Default  28.0000  28.0000
1148        blend  13537.000000  13537.000000  Default  27.0000  27.0000
5293         good  18872.000000  18872.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
11610  strawberry   1340.472531   3470.728019  Topic10  -4.8548   2.0461
4779     followed   1260.878110   3336.443396  Topic10  -4.9160   2.0244
942     beautiful   1066.524997   2716.454215  Topic10  -5.0834   2.0625
1148        blend   1357.899748  13537.203112  Topic10  -4.8419   0.6980
6739       length    983.367225   2133.850586  Topic10  -5.1646   2.2228

[600 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
64         8  0.999127  accessible
101        7  0.999579      acidic
105        1  0.020702     acidity
105        2  0.368418     acidity
105        3  0.303273     acidity
...      ...       ...         ...
13648      3  0.997865       youth
13671      2  0.936065        zest
13671      7  0.063631        zest
13679      4  0.927148         zin
13679      8  0.072384         zin

[890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 9, 1, 7, 6, 2, 3, 4, 10])

In [ ]:
print("Perplexity: ", lda.perplexity(data_vectorized)) #lower the better

Perplexity:  1305.8252027864442
